In [0]:
from google.colab import drive
import warnings
import pandas as pd
import numpy as np
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'
warnings.filterwarnings('ignore')
fasttext_data = pd.read_csv('2020_05_09_encoded_Fasttext.csv', index_col=0)
print(fasttext_data.shape)
base_data = pd.read_csv('papildinatie_jautajumi_cleaned.csv', index_col=0)
print(base_data.shape)
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'
bert_big_data = pd.read_csv('2020_05_10_encoded_BERT_big.csv', index_col=0)
bert_multi_cased_data = pd.read_csv('2020_05_10_encoded_BERT_multi_cased.csv', index_col=0)
bert_small_2_data = pd.read_csv('2020_05_10_encoded_BERT_small_2.csv', index_col=0)
bert_small_data = pd.read_csv('2020_05_10_encoded_BERT_small.csv', index_col=0)
print(bert_big_data.shape, bert_multi_cased_data.shape, bert_small_2_data.shape, bert_small_data.shape)
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Rezultāti'

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.preprocessing import text
import matplotlib.pyplot as plt
from keras import layers
from keras import models
import datetime
from imblearn.under_sampling import ClusterCentroids, NearMiss, RandomUnderSampler, CondensedNearestNeighbour, EditedNearestNeighbours, InstanceHardnessThreshold, TomekLinks
from imblearn.base import BaseSampler

In [0]:
class FakeSampler(BaseSampler):
    _sampling_type = 'bypass'
    def _fit_resample(self, X, y):
        return X, y
def transformResults(results):
    mod_results = []
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)  
    return mod_results 

In [0]:
oversampler  = FakeSampler()
indices       = range(6)
dataset_list = [bert_small_2_data, bert_small_data, bert_multi_cased_data, bert_big_data, fasttext_data]
model_names  = ['BERT small #2', 'BERT small', 'BERT multi cased', 'BERT big', 'FastText']
n_split = 5
counter = 0 
results1 = []
header_row = ['Modelis', 'RandomUnderSampler', 'NearMiss', 'ClusterCentroids', 'TomekLinks', 'EditedNearestNeighbours', 'InstanceHardnessThreshold']
results1.append(header_row)

<p>1. CondensedNeearestNeighbors nesniedza nekādu rezultātu apmēram 10min laikā, tika pārtraukta un izņemta no eksperimentiem.<br>
2. EditedNearestNeighbours izmeta visus klases 2 ierakstus<br></p>

In [0]:
exec_start = datetime.datetime.now()
for dataset in dataset_list:
  result_row = []
  result_row.append(model_names[counter])
  # Extracting the features and the target based on dimensionality of each dataset
  col_count = len(dataset.columns)
  features = dataset.iloc[:,:col_count-1]
  target = dataset.iloc[:,col_count-1:]

  features = features.to_numpy()
  target = target.to_numpy()

  for i, oversampler in zip( indices, (RandomUnderSampler(),
                                       NearMiss(),
                                       ClusterCentroids(),
                                       #CondensedNearestNeighbour(),
                                       TomekLinks(),
                                       EditedNearestNeighbours(),
                                       InstanceHardnessThreshold()
                                        )):
    os_name = oversampler.__class__.__name__
    date_start    = datetime.datetime.now()                                     # Fiksējam Unersampling sākuma laiku     
    
    date_end = datetime.datetime.now()                                          # Fiksējam beigu laiku
    time_elapsed_us = date_end - date_start                                     # Fiksējam UnderSampling patērēto laiku
    unique, counts = np.unique(y_res, return_counts=True)
    values = np.asarray((unique, counts)).T
    print('UnderSampled dataset structure with', os_name, values)

    model_f1_scores = []
    fold_counter = 0
    for train_index,test_index in StratifiedKFold(n_split).split(X_res, y_res):
      # creating train and test datasets
      x_train,x_test=X_res[train_index],X_res[test_index]
      y_train,y_test=y_res[train_index],y_res[test_index]  
      # Firstly, creating the neural network model
      model = models.Sequential()
      model.add(layers.Dense(256, activation='relu', input_shape=(col_count-1,)))
      model.add(layers.Dense(64, activation='relu'))
      model.add(layers.Dense(4, activation='softmax'))
      # Then, compiling the created model
      model.compile(loss=['sparse_categorical_crossentropy'],
                    optimizer='adam',
                    metrics=['sparse_categorical_accuracy'])
      batch_size = 64
      epochs = 5
      # Then, fitting the model
      history = model.fit(x_train, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=0)
      y_pred = model.predict(x_test)                                                  # Prasam modelim paredzēt testa datiem kategorijas
      y_pred2 = transformResults(y_pred)                                              # Pārveidojam rezultātus

      model_f1_avg = round(f1_score(y_test, y_pred2, average='macro'),4)              # Fiksējam f1 vidējo rezultātu kārtējā K-Foldā
      model_f1_scores.append(model_f1_avg)
      fold_counter = fold_counter + 1
      print('\tCompleted for model: ',model_names[counter],' K-Fold #', fold_counter, 'F1 score:', model_f1_avg)
  
    #After the K-Fold validation
    model_f1 = round(np.mean(model_f1_scores), 4)
    result_row.append(model_f1)
    print('Completed for:', os_name,' with F1 score:',model_f1, ' Oversampling duration:',time_elapsed_us)  
  #After all four UnderSamplers
  results1.append(result_row)
  print('Completed for ', model_names[counter], ' with results=', result_row)
  counter = counter + 1

#In the very end - fix the execution time
exec_end = datetime.datetime.now()
exec_elapsed = exec_end - exec_start
print('Total time elapsed:', exec_elapsed)

# Saglabājam visus rezultātus
result_data = pd.DataFrame(results1[1:], columns=results1[0])
result_data.to_csv('2020_05_16_UnderSampling_results.csv')
result_data

UnderSampled dataset structure with RandomUnderSampler [[  0 122]
 [  1 122]
 [  2 122]
 [  3 122]]
	Completed for model:  BERT small #2  K-Fold # 1 F1 score: 0.6326
	Completed for model:  BERT small #2  K-Fold # 2 F1 score: 0.597
	Completed for model:  BERT small #2  K-Fold # 3 F1 score: 0.7068
	Completed for model:  BERT small #2  K-Fold # 4 F1 score: 0.674
	Completed for model:  BERT small #2  K-Fold # 5 F1 score: 0.6012
Completed for: RandomUnderSampler  with F1 score: 0.6423  Oversampling duration: 0:00:00.011211
UnderSampled dataset structure with NearMiss [[  0 122]
 [  1 122]
 [  2 122]
 [  3 122]]
	Completed for model:  BERT small #2  K-Fold # 1 F1 score: 0.4802
	Completed for model:  BERT small #2  K-Fold # 2 F1 score: 0.7349
	Completed for model:  BERT small #2  K-Fold # 3 F1 score: 0.7722
	Completed for model:  BERT small #2  K-Fold # 4 F1 score: 0.7149
	Completed for model:  BERT small #2  K-Fold # 5 F1 score: 0.6792
Completed for: NearMiss  with F1 score: 0.6763  Oversamp

,Modelis,RandomUnderSampler,NearMiss,ClusterCentroids,TomekLinks,EditedNearestNeighbours,InstanceHardnessThreshold
0,BERT small #2,0.6423,0.6763,0.6214,0.4663,0.7136,0.7109
1,BERT small,0.7701,0.7662,0.7968,0.6069,0.6153,0.7626
2,BERT multi cased,0.7652,0.7981,0.7332,0.5605,0.7759,0.7466
3,BERT big,0.8274,0.8400,0.9063,0.6391,0.7917,0.8598
4,FastText,0.7791,0.8788,0.8391,0.6796,0.6447,0.8435
